# Monitoring forest clearfelling in Wales

## Background

### Clearfelling in Wales
Wales is one of the least wooded countries in Europe, with woodland covering only 15% of the land area, compared to the European average of 38% (Welsh Government, 2018). In 2018, Welsh Government published the latest version of Woodlands for Wales.The 50-year strategy outlines Welsh Government’s vision and targets for Wales’ trees, woodlands and forest and aims to ensure high-quality woodlands that: 
- enhance the landscape,
- are appropriate to local conditions,
- have a diverse mixture of species and habitats.

In 2017 (Natural Resources Wales, 2017), the vast majority of managed woodlands were reported as:
- planted single species, 
- single aged (over half is coniferous),
- predominantly manged on clearfell & restock regime. 



To increase the diversity of woodlands, the range of silvicultural systems currently used by managers needs to widen.
Typically, forests managed solely by a clearfell silvicultural system are characterised by poor structural diversity with uniform aged crops, limited or no diversity of structure within a stand and sometimes whole forest blocks using
the same system. In contrast, forests managed using one or more of the range of Low Impact Silvicultural Systems (LISS) tend to be more structurally diverse, and therefore more resilient.

One of the objectives of the Woodlands for Wales strategy is to reduce the amount of clearfelling and increase the use of LISS. Therefore, monitoring clearfelling and condition of forests is important to show sucess of the Woodlands for Wales strategy.

![Image of Yaktocat](https://i.guim.co.uk/img/media/08dd74abf49c450818eae10853155cc6ae450604/0_0_6000_4000/master/6000.jpg?width=700&quality=85&auto=format&fit=max&s=545b3dbc65ccb322efef78a35c6c779e)
         
                  Clearfelled Sitka spruce. Photograph: Tom Allan

## Description
This notebook demonstrates how the new Wales Data Cube allows to quickly analyse multiple years of nearly daily Sentinel-1 Synthetic Aperture Radar (SAR) data, to monitor forest clearfelling. 

Contrary to Optical sensors, which are maily sensitive to colour and chemistry, SAR data are stronly correlated to height and texture. Moreover, SAR data have the advantage of operating at wavelengths not impeded by cloud cover, illumination or weather conditions, allowing to monitor forests during the whole year, including during winter period. 

Living Wales project has allowed generation of national annual land cover maps (2017-2021), and is also actively involved in the development of a new digital infrastructure (incl. Open Data Cube technology) for Wales that provides Analysis Ready Data, which are satellite data that have been processed to a minimum set of requirements and organized into a form that allows immediate analysis with minimum additional user effort, and derived products.

This notebook uses the Analysis Ready Sentinel-1 Data, as well as a python library for forest and clearfelling mapping, developed and provided by the Living Wales project.


Topics include: 

1. Querying data for an area and period of interest (i.e., 01 Jan 2017 - 12 Dec 2021)
2. Loading data from datacube
3. Mapping forest areas
4. Mapping annual clearfells
5. Mapping the extent and date of clearfells for the whole period
6. Automatised reporting on clearfells for a period and area of interest



### Jupyter Notebooks
#### Running (executing) a cell
Jupyter Notebooks allow code to be separated into sections that can be executed independent of one another.
These sections are called "cells".

Python code is written into individual cells that can be executed by placing the cursor in the cell and typing `Shift-Enter` on the keyboard or selecting the &#9658; "Run" button in the ribbon at the top of the notebook.
These options will run a single cell at a time.

To automatically run all cells in a notebook, navigate to the "Cell" tab of the menu bar at the top of JupyterLab and select "Run All" (or the option that best suits your needs).
When a cell is run, the cell's content is executed.
Any output produced from running the cell will appear directly below it.

Run the cell below as a test:

In [ ]:
print("I ran a cell!")

#### Cell status
The `[ ]:` symbol to the left of each Code cell describes the state of the cell:

* `[ ]:` means that the cell has not been run yet.
* `[*]:` means that the cell is currently running.
* `[1]:` means that the cell has finished running and was the first cell run. The number indicates the order that the cells were run in.

***

## Getting started

To run this analysis, run all the cells in the notebook starting with the 'Load packages and connect to the datacube' cell.

### Load packages

Load key Python packages and supporting functions for the analysis, then connect to the datacube. 

In [ ]:
import sys
sys.path.append("..")

import datacube
import matplotlib.pyplot as plt
from time import time as time
import warnings
warnings.filterwarnings("ignore")
import wales_utils.data_cube_utilities.display_tools as display_tools


# Import Living Wales 'forest' Python library
import wales_utils.data_cube_utilities.forest as forest

### Connect to the datacube
Connect to the datacube so we can access Living Wales Analysis Ready Data. 

In [ ]:
dc = datacube.Datacube()

### Query an area and period
#### Area :  Please choose one of the two Tywi Forest sites, enter the name in the cell below and "Run" it
- 'esgair gelli'
- 'cefn fannog'

In [ ]:
######################
#  ENTER FOREST NAME #
######################

site_name = 'esgair gelli'

In [ ]:
# select the above site
site = forest.select_forest_site(site_name)

#### Display the extent of the selected forest site on ESRI World Imagery
World Imagery provides very high resolution (one meter or better) satellite and aerial imagery in many parts of the world. 
However, it is static imagery and, as a consequence, for each place there is only one date is available. 


For the study sites available this notebook, it is a Maxar (Vivid) image captured 2 years 3 months ago, on 26 Mar 2020, with a 0.50 meters resolution. 


This image is only used for visualising the extent of the selected study site and is not used in the following analyses.

In [ ]:
display_tools.map_extent(site)

#### Period
For this example, period of interest has been fixed to five years (01 Jan 2017 to 31 Dec 2021).

In [ ]:
start_date = '2017-01-01'
end_date = '2021-12-31'

print("Analysed period: " + start_date + " to " + end_date)

### Loading data from datacube 

#### Load data

In [ ]:
# Preparing query for datacube
start_time = time()
query = forest.query_site_period(site, start_date, end_date)

# Let's load and clean a demo dasatet
dataset_in = dc.load(**query,
                    dask_chunks={'x': 1000, 'y': 1000})

# define nodata
dataset_in = dataset_in.where(dataset_in != 0)
# drop dates with only nodata for the area of interest
dataset_in = dataset_in.dropna('time', how='all')
# rename x and y with explicit names
dataset_in = dataset_in.rename({'x': 'longitude', 'y': 'latitude'})

print("Datacube ready")
print("Took only " + str(round(time()-start_time,2)) + " seconds to load 5 years of almost daily data.")

#### Have a look at the dataset structure

In [ ]:
dataset_in

#### Groupping images by month for each year

In [ ]:
S1_dataset = forest.group_data_by(dataset_in, "month")

### Mapping forested areas
In this section, we will create a forest map for each year of the period of interest using the Living Wales python library, and visualise them.

In [ ]:
# Map forest areas for each year using Sentinel-1 data
woody = forest.forest_mapping(S1_dataset)


# Visualise forest extent for each year
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds)")

woody.plot(col="year", col_wrap=3, add_colorbar=False, size=5, cmap="Greens")

### Mapping annual clearfells
In this section, we will be detecting and mapping annual clearfells during the period of interest using the Living Wales python library.


In [ ]:
# Detecting clearfells in forest for each year
clear_cuts = forest.clearfells_monitoring(woody)

# Visualise forest clearfells for each year
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds)")

clear_cuts.plot(col="year", col_wrap=2, add_colorbar=False, size=4, cmap="Reds")

### Mapping the extent and date of clearfells for the whole period
In this section, we will be mapping the extent and year of the clearfells. 

- CYAN:      2021 clearfell
- PINK:      2020 clearfell
- RED:       2019 clearfell
- DARK BLUE: 2018 clearfell

In [ ]:
# Retrieving date of clearfells
clearfell_date = forest.mapping_clearfelling_dates(clear_cuts)

# Visualise date and extent of forest clearfells
print("Plotting ...")
print("(Please wait until images appear. This may take a few seconds)")

clearfell_date.plot(figsize=(7,6), cmap='tab10', add_colorbar=False)
plt.title('Date and extent of cleafells')

### Automatised reporting on clearfells for the period and area of interest

In [ ]:
forest.clearfell_reporting(site_name, clearfell_date)